<a href="https://colab.research.google.com/github/maberf/colabs/blob/main/Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import os
from google.colab import drive
from google.colab import auth
from google.auth import default
import gspread

In [ ]:
# Stock tickers - Insert here
tickers = ['^BVSP','ARZZ3.SA','AURE3.SA','B3SA3.SA','BBAS3.SA','BBDC4.SA','BBSE3.SA','CIEL3.SA','CPFE3.SA','CSAN3.SA','EGIE3.SA','GGBR4.SA','ITSA4.SA','JALL3.SA','JHSF3.SA','KLBN11.SA','LEVE3.SA','LREN3.SA','ODPV3.SA','PETR4.SA','RAIZ4.SA','STBP3.SA','TAEE11.SA','VALE3.SA','VIVT3.SA']

In [ ]:
# Real State Invesments Trust tickers - Insert here
# Do not insert IFIX here, yahoo finance return IFIX.SA maximum 5 days. IFIX should be calculated from an csv file with 1 year history ffrom Investing.com and upload in personal Google Drive
tickerr = ['BTLG11.SA','HGCR11.SA','HGBS11.SA','HGRE11.SA','HGRU11.SA','HSLG11.SA','HSML11.SA','HTMX11.SA','JSAF11.SA','JFLL11.SA','KNCA11.SA','KNHF11.SA','KNIP11.SA','LUGG11.SA','MALL11.SA','MFII11.SA','SADI11.SA','TGAR11.SA','TRXF11.SA','VGHF11.SA','VILG11.SA','VISC11.SA','XPLG11.SA','XPML11.SA'] # Do not insert IFIX here, yahoo finance return IFIX.SA maximum 5 days

In [ ]:
# Load tickers history prices in a dataframe considering a certain period of time
dfs = yf.download(tickers, period='2y')['Close']
dfr = yf.download(tickerr, period='1y')['Close']
# display(dfs)
# display(dfr)

[*********************100%%**********************]  25 of 25 completed
[*********************100%%**********************]  24 of 24 completed


In [ ]:
# IFIX historic series from Investing.com to be appended in real state dataframe dfr - https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data
# Google Drive mounth
drive.mount('/content/drive', force_remount=True)
# File path on Google Drive - Download the file and upload to Financas folder in Google Drive. Rename the path according file name uploaded.
ifixfile = '/content/drive/MyDrive/Financas/history.csv'
# File csv to dataframe converting quote to float
ifix = pd.read_csv(ifixfile, thousands = '.', decimal = ',', dtype = {'Último':np.float64})
# Excluding and rename columns
ifix = ifix.drop(columns=['Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%'])
ifix = ifix.rename(columns={'Data': 'Date', 'Último': 'IFIX'})
# Date format in Date column
ifix['Date'] = pd.to_datetime(ifix['Date'], format='%d%m%Y')
# display(ifix)

Mounted at /content/drive


In [ ]:
# Append ifix dataframe in dfr dataframe by key = date
dfr = dfr.merge(ifix, how='inner', on='Date')
# display(dfr)

In [ ]:
# Calculate daily variation
dfs[tickers] = np.log(dfs[tickers]/dfs[tickers].shift(1))
tickerr.append('IFIX') # now 'IFIX' value can be inserted on the tickers list
dfr[tickerr] = np.log(dfr[tickerr]/dfr[tickerr].shift(1))

In [ ]:
# Exclude NaNs in the first row
dfs.dropna(inplace=True)
dfr.dropna(inplace=True)
# display(dfs)
# display(dfr)

In [ ]:
# Market return variance calculation
vars = dfs['^BVSP'].var()
varr = dfr['IFIX'].var()

In [ ]:
# Covariance calculation
covs = dfs.cov()
covr = dfr.cov()

In [ ]:
# Beta calculation
betas = covs['^BVSP']/vars
betas = betas.round(2)
betar = covr['IFIX']/varr
betar = betar.round(2)

In [ ]:
# Dataframe beta creation
betas = betas.to_frame()
betar = betar.to_frame()
# sorting beta alhabetical ascending order by ticker
betas = betas.sort_index()
betar = betar.sort_index()
# Date has been changed to a value in index and has to be removed
betar = betar.drop(index='Date')
# beta = beta[::-1]
# display(betas)
# display(betar)

In [ ]:
# FORMAT BETA DATAFRAME TO SPREADSHEET WRITING
#
# Creating an empty column "index"
betas.insert(1,'Index','')
betar.insert(1,'Index','')
betas = betas.set_axis(['Beta', 'Index'], axis=1)
betar = betar.set_axis(['Beta', 'Index'], axis=1)
# display(betas)
# display(betar)

In [ ]:
# Filling Index column with row values
for i in range(0, int(len(betas))):
   betas['Index'].iloc[i] = str(i)
   i = i + 1
for i in range(0, int(len(betar))):
   betar['Index'].iloc[i] = str(i)
   i = i + 1

<ipython-input-569-32979ba70b13>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betas['Index'].iloc[i] = str(i)
<ipython-input-569-32979ba70b13>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betas['Index'].iloc[i] = str(i)
<ipython-input-569-32979ba70b13>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betas['Index'].iloc[i] = str(i)
<ipython-input-569-32979ba70b13>:3: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [ ]:
# Organizing columns order and making Index column as index of dataframe
betas['Ticker'] = betas.index # create Ticker as data column
betar['Ticker'] = betar.index # create Ticker as data column
# set "Index" column as index
betas = betas.set_index('Index')
betar = betar.set_index('Index')
# Columns rename
betas= betas.reindex(['Ticker', 'Beta'], axis=1)
betar= betar.reindex(['Ticker', 'Beta'], axis=1)
# display(betas)
# display(betar)

In [ ]:
# excluding .SA, rename value market return and ascending order
betas['Ticker'] = betas['Ticker'].str.replace('.SA', '')
betar['Ticker'] = betar['Ticker'].str.replace('.SA', '')
betas['Ticker'] = betas['Ticker'].str.replace('^BVSP', 'IBOV')
betas = betas.sort_values(by='Ticker')
betar = betar.sort_values(by='Ticker')
# display(betas)
# display(betar)

In [ ]:
# Autentication in Google Docs (only once)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Open workbook and worksheets
wb = gc.open('Quotes')
wss = wb.worksheet('BetaS')
wsr = wb.worksheet('BetaR')

In [ ]:
# Write data in the worksheets
wss.update([betas.columns.values.tolist()] + betas.values.tolist())
wsr.update([betar.columns.values.tolist()] + betar.values.tolist())

{'spreadsheetId': '1qgTSxri55kYWVahW6sH3Fbn3ofWzhq93umUJhcwO7Uk',
 'updatedRange': 'BetaR!A1:B26',
 'updatedRows': 26,
 'updatedColumns': 2,
 'updatedCells': 52}